<a href="https://colab.research.google.com/github/alaukikpachauri/parkinsons-updrs-predictor-/blob/main/ugh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install gradio

In [21]:
import kagglehub
import pandas as pd


path= kagglehub.dataset_download("leilahasan/parkinson-dataset")

print("Path to dataset files:", path)

data = pd.read_csv(path + "/parkinsons_updrs.csv")

data.head()

Using Colab cache for faster access to the 'parkinson-dataset' dataset.
Path to dataset files: /kaggle/input/parkinson-dataset


,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os


# Split the data into train, test, and validation sets
# Assuming an 80-10-10 split for train, validation, and test respectively
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Display basic information about the loaded data
print("Train Data Shape:", train_data.shape)
print("Validation Data Shape:", validation_data.shape)
print("Test Data Shape:", test_data.shape)

Train Data Shape: (4758, 22)
Validation Data Shape: (529, 22)
Test Data Shape: (588, 22)


In [23]:
from sklearn.preprocessing import StandardScaler

# Separate features and target variables
X_train = train_data.drop(columns=['total_UPDRS', 'motor_UPDRS'])
y_total_updrs = train_data['total_UPDRS']
y_motor_updrs = train_data['motor_UPDRS']

# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val = validation_data.drop(columns=['total_UPDRS', 'motor_UPDRS'])
X_test = test_data.drop(columns=['total_UPDRS', 'motor_UPDRS'])

# Scale validation and test sets using the same scaler fitted on the training data
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [24]:
# Calculate mean and standard deviation of scaled features
mean_scaled = X_train_scaled.mean(axis=0)
std_scaled = X_train_scaled.std(axis=0)

print("Mean of scaled features:", mean_scaled)
print("Standard deviation of scaled features:", std_scaled)


Mean of scaled features: [ 8.96018582e-18 -3.04646318e-16  3.95741541e-17 -1.22455873e-16
 -6.86947580e-17  8.81084939e-17 -1.49336430e-18 -2.09071003e-17
  2.09071003e-17  4.40542470e-17 -2.98672861e-17  8.06416724e-17
  9.63219976e-17  3.14353186e-16  4.85343399e-17 -1.49336430e-17
  2.02350863e-16  1.07522230e-16  1.04908842e-15 -2.98672861e-16]
Standard deviation of scaled features: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [25]:
# Create a DataFrame from the scaled features
scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

# Display summary statistics of scaled features
print(scaled_df.describe())


           subject#           age           sex     test_time     Jitter(%)  \
count  4.758000e+03  4.758000e+03  4.758000e+03  4.758000e+03  4.758000e+03   
mean   8.960186e-18 -3.046463e-16  3.957415e-17 -1.224559e-16 -6.869476e-17   
std    1.000105e+00  1.000105e+00  1.000105e+00  1.000105e+00  1.000105e+00   
min   -1.659513e+00 -3.306728e+00 -6.831601e-01 -1.822146e+00 -9.245760e-01   
25%   -9.320726e-01 -7.875316e-01 -6.831601e-01 -8.663732e-01 -4.536133e-01   
50%    3.784825e-02  1.403083e-02 -6.831601e-01 -1.003320e-02 -2.155348e-01   
75%    9.269424e-01  8.155933e-01  1.463786e+00  8.631211e-01  1.091178e-01   
max    1.654383e+00  2.304209e+00  1.463786e+00  2.286794e+00  1.624478e+01   

        Jitter(Abs)    Jitter:RAP   Jitter:PPQ5    Jitter:DDP       Shimmer  \
count  4.758000e+03  4.758000e+03  4.758000e+03  4.758000e+03  4.758000e+03   
mean   8.810849e-17 -1.493364e-18 -2.090710e-17  2.090710e-17  4.405425e-17   
std    1.000105e+00  1.000105e+00  1.000105e+00  1.

In [26]:
from sklearn.preprocessing import MinMaxScaler

#7
# Select numerical columns
numerical_columns = ['age', 'sex', 'test_time', 'Jitter(%)', 'Jitter(Abs)', 'Jitter:RAP',
                     'Jitter:PPQ5', 'Jitter:DDP', 'Shimmer', 'Shimmer(dB)', 'Shimmer:APQ3',
                     'Shimmer:APQ5', 'Shimmer:APQ11', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'PPE']

# Exclude 'total_UPDRS', 'motor_UPDRS', 'subject' from numerical columns
numerical_columns = [col for col in numerical_columns if col not in ['total_UPDRS', 'motor_UPDRS', 'subject']]

# Separate numerical columns
numerical_data = data[numerical_columns]

# Scale numerical columns using MinMaxScaler
scaler = MinMaxScaler()
numerical_data_normalized = scaler.fit_transform(numerical_data)

# Create a DataFrame with the normalized numerical columns
numerical_data_normalized_df = pd.DataFrame(numerical_data_normalized, columns=numerical_columns)

# Display the normalized numerical data
print(numerical_data_normalized_df.head())


        age  sex  test_time  Jitter(%)  Jitter(Abs)  Jitter:RAP  Jitter:PPQ5  \
0  0.734694  0.0   0.045076   0.058390     0.071164    0.064324     0.039635   
1  0.734694  0.0   0.077034   0.021884     0.032819    0.017305     0.015478   
2  0.734694  0.0   0.108957   0.040137     0.050458    0.030065     0.023868   
3  0.734694  0.0   0.136105   0.044877     0.054856    0.027618     0.031969   
4  0.734694  0.0   0.172487   0.025413     0.040353    0.010488     0.012585   

   Jitter:DDP   Shimmer  Shimmer(dB)  Shimmer:APQ3  Shimmer:APQ5  \
0    0.064433  0.085062     0.098030      0.079287      0.067543   
1    0.017303  0.064691     0.073522      0.051720      0.053186   
2    0.030178  0.051549     0.074483      0.035577      0.039375   
3    0.027673  0.075423     0.144642      0.058674      0.064878   
4    0.010486  0.052604     0.072081      0.032162      0.044524   

   Shimmer:APQ11  Shimmer:DDA       NHR       HNR      RPDE       DFA  \
0       0.051764     0.079267  0.0187

In [36]:
import pandas as pd

# Assuming X_train contains your feature columns and y_total_updrs, y_motor_updrs are your target variables
# Concatenate the features and target variables into one DataFrame
df = pd.concat([X_train, y_total_updrs, y_motor_updrs], axis=1)

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Print the correlation of features with 'total_UPDRS'
print("Correlation with total_UPDRS:")
print(correlation_matrix['total_UPDRS'].sort_values(ascending=False))

# Print the correlation of features with 'motor_UPDRS'
print("\nCorrelation with motor_UPDRS:")
print(correlation_matrix['motor_UPDRS'].sort_values(ascending=False))


Correlation with total_UPDRS:
total_UPDRS      1.000000
motor_UPDRS      0.946673
age              0.311348
RPDE             0.155770
PPE              0.154181
Shimmer:APQ11    0.116928
Shimmer(dB)      0.094467
Shimmer          0.089161
Shimmer:APQ5     0.080039
Name: total_UPDRS, dtype: float64

Correlation with motor_UPDRS:
motor_UPDRS      1.000000
total_UPDRS      0.946673
age              0.274735
PPE              0.160303
Shimmer:APQ11    0.131447
RPDE             0.127226
Shimmer(dB)      0.104575
Shimmer          0.097976
Shimmer:APQ5     0.087458
Name: motor_UPDRS, dtype: float64


In [29]:
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



# Features selected based on the earlier analysis
selected_features = ['age', 'PPE', 'Shimmer:APQ11', 'RPDE', 'Shimmer(dB)', 'Shimmer', 'Shimmer:APQ5']

# Separate features and target variables
X_train = data[selected_features]
y_train = data[['total_UPDRS', 'motor_UPDRS']]

# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Initialize a MultiOutput Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Create a MultiOutput Regressor from Random Forest
multi_output_rf = MultiOutputRegressor(rf)

# Fit the model
multi_output_rf.fit(X_train_scaled, y_train)

# Predict on the training set (just as an example)
predictions_train = multi_output_rf.predict(X_train_scaled)

# Evaluate the model on training data (just as an example)
mae_train = mean_absolute_error(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)
r2_train = r2_score(y_train, predictions_train)

print(f"Training Mean Absolute Error: {mae_train}")
print(f"Training Mean Squared Error: {mse_train}")
print(f"Training R^2 Score: {r2_train}")





Training Mean Absolute Error: 1.1142852344472467
Training Mean Squared Error: 2.736953036858722
Training R^2 Score: 0.9693757273021606


In [32]:
# Assuming you already have the model, scaler, X_test, and y_test from previous code snippets

# Select only the features that the model was trained on for the test set
X_test_filtered = X_test[selected_features]

# Scale the test features using the same scaler used for training data
X_test_scaled = scaler.transform(X_test_filtered)

# Predict on the test set
predictions_test = multi_output_rf.predict(X_test_scaled)

# Extract y_test (total_UPDRS and motor_UPDRS) from the original test_data
y_test = test_data[['total_UPDRS', 'motor_UPDRS']]

# Evaluate the model on test data
mae_test = mean_absolute_error(y_test, predictions_test)
mse_test = mean_squared_error(y_test, predictions_test)
r2_test = r2_score(y_test, predictions_test)

print(f"Test Mean Absolute Error: {mae_test}")
print(f"Test Mean Squared Error: {mse_test}")
print(f"Test R^2 Score: {r2_test}")

Test Mean Absolute Error: 3.1059894064625846
Test Mean Squared Error: 20.551649499952113
Test R^2 Score: 0.7502844373160592


In [33]:
# Example input features (replace these values with your own)
input_features = {
    'age': 60,
    'PPE': 0.25,
    'Shimmer:APQ11': 0.05,
    'RPDE': 0.4,
    'Shimmer(dB)': 0.8,
    'Shimmer': 0.06,
    'Shimmer:APQ5': 0.03
}

# Reshape the input data to a format that the model expects
input_data = []
for feature in selected_features:
    input_data.append(input_features[feature])

# Reshape the input data to match the model's input shape
input_data = [input_data]

# Use the trained model to make predictions
predictions = multi_output_rf.predict(input_data)

# Print the predicted values
predicted_total_updrs, predicted_motor_updrs = predictions[0]
print("Predicted Total UPDRS:", predicted_total_updrs)
print("Predicted Motor UPDRS:", predicted_motor_updrs)


Predicted Total UPDRS: 26.23832999999999
Predicted Motor UPDRS: 16.19282999999999


In [35]:

import gradio as gr

# Define your function to predict based on the model
def predict_total_motor_updrs(age, PPE, Shimmer_APQ11, RPDE, Shimmer_dB, Shimmer, Shimmer_APQ5):
    input_features = [[age, PPE, Shimmer_APQ11, RPDE, Shimmer_dB, Shimmer, Shimmer_APQ5]]
    input_features = scaler.transform(input_features)
    # Use your model to predict total and motor UPDRS scores
    predictions = multi_output_rf.predict(input_features)
    return predictions[0][0], predictions[0][1]

# Define your Gradio interface
iface = gr.Interface(
    fn=predict_total_motor_updrs,
    inputs=["number", "number", "number", "number", "number", "number", "number"],
    outputs=["label", "label"],
    title="Parkinson's UPDRS Predictor",
    description="Enter feature values to predict Total and Motor UPDRS scores."
)

# Launch the Gradio interface
iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a1d1d99002d2e437c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
